<a href="https://colab.research.google.com/github/Alejandro-sin/Learning-Path-to-AI/blob/master/LLM_OpenSource_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hugging Faces


Huggin Faces es el sitio por excelencia para la inteligencia artificial opensource, es un espacio que sirve de repositorio de modelos, datasets, y espacios dedicados a la inferencia.



In [1]:
!pip install -q transformers datasets diffusers

## Setup

In [2]:
# Use a pipeline as a high-level helper


## AUTH
from google.colab import userdata
from huggingface_hub import login


# TORCH - Diffsuser
import torch
from diffusers import FluxPipeline
from transformers import pipeline
from diffusers import DiffusionPipeline
from datasets import load_dataset

# Display Image and Sound
import soundfile as sf
from IPython.display import Audio, display
from PIL import Image



In [3]:
hf_token = userdata.get('HF')
login(hf_token, add_to_git_credential=True)

## Pipelines


Los pipelines en HugginFace proveen una interfaz de alto nivel para operar con los modelos, y realizar inferencias con los modelos ya entrenados. La idea es revisar algunas de las tareas que podemos hacer con los transformers.

- Analisis de Sentimientos
- Named Entity Recognition
- Preguntas y respuestas con contexto
- Resumen de texto
- Traducción y clasificaión
- Generación de imagenes, audio y video.
- Feature Extraction


### Anlisis de Sentimiento.

In [ ]:
classifier = pipeline("sentiment-analysis", device="cuda")
result = classifier("""Rainer Maria Rilke is considered one of the most lyrically intense German-language poets.
            His work spans the late 19th and early 20th centuries, bridging the gap between the traditional Romantic era and the rising tide of Modernism.""")
print(result)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda


## NER

La extracción de entidades es una tarea muy común en las operaciones de lenguaje natural, buscamos etiquetar textos condiferentes TAGs asociados a la entidad. Persoans, organizaciones, lugares y demás.

- Si no pasamos un modelo en concreto lo que sucederá es que se usará uno por defecto, en este caso se usó el modelo por defecto para texto en inglés `dbmdz/bert-large-cased-finetuned-conll03-english`

- Podemos usar ejemplos más especializados, por ejemplo para contextos medicos donde el modelo se finetuneo para reconocer estas entidades. `blaze999/Medical-NER`

In [ ]:
ner = pipeline("ner", grouped_entities=True, device="cuda")
result = ner("""Barack Obama was the 44th president of the United States""")


In [ ]:
print(result)

In [ ]:
ner = pipeline("token-classification", model="blaze999/Medical-NER")
result = ner("""A 48 year-old female presented with vaginal bleeding and abnormal Pap smears.
Upon diagnosis of invasive non-keratinizing SCC of the cervix, she underwent a radical hysterectomy with salpingo-oophorectomy
which demonstrated positive spread to the pelvic lymph nodes and the parametrium.
Pathological examination revealed that the tumour also extensively involved the lower uterine segment.""")
print(result)

## Preguntas y Respuestas

In [ ]:
question_answerer = pipeline("question-answering", device="cuda")
result = question_answerer(question="Who was the 44th president of the United States?", context="Barack Obama was the 44th president of the United States.")
print(result)

## Resumenes

In [ ]:
summarizer = pipeline("summarization", device="cuda")
text = """A knowledge graph, also known as a semantic network, represents a network of real-world entities—such as objects, events,
situations or concepts—and illustrates the relationship between them.
This information is usually stored in a graph database and visualized as a graph structure, prompting the term knowledge “graph.”
"""
summary = summarizer(text, max_length=50, min_length=25, do_sample=False)
print(summary[0]['summary_text'])

Podemos usar generadores también para producir los resultados

In [ ]:
text = """A knowledge graph, also known as a semantic network, represents a network of real-world entities—such as objects, events,
    situations or concepts—and illustrates the relationship between them.
    This information is usually stored in a graph database and visualized as a graph structure, prompting the term knowledge “graph.”
"""
summarizer = pipeline("summarization", device="cuda")

def pipeline_generator(text):
  '''
  Función sencilla para resumir texto.

  '''
  sentinel = 0
  while True:
    summary = summarizer(text, max_length=50, min_length=25, do_sample=False)
    if sentinel == 1:
      break
    sentinel += 1
    yield summary[0]['summary_text']

for n in pipeline_generator(text):
  print(n)

## Traducción

In [ ]:
translator = pipeline("translation_en_to_fr", device="cuda")
result = translator("The Data Scientists were truly amazed by the power and simplicity of the HuggingFace pipeline API.")
print(result[0]['translation_text'])

In [ ]:
# All translation models are here: https://huggingface.co/models?pipeline_tag=translation&sort=trending

translator = pipeline("translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es", device="cuda")
result = translator("The Data Scientists were truly amazed by the power and simplicity of the HuggingFace pipeline API.")
print(result[0]['translation_text'])

In [ ]:
# Reutilicemos el texto de los grafos de conocimiento para clasificar.
classifier = pipeline("zero-shot-classification", device="cuda")
result = classifier(text, candidate_labels=["technology", "sports", "politics"])
print(result)

### Generación de Imagenes


In [ ]:
if 1==1:
  image_gen = DiffusionPipeline.from_pretrained(
      "stabilityai/stable-diffusion-2",
      torch_dtype=torch.float16,
      use_safetensors=True,
      variant="fp16"
      ).to("cuda")

  #text = "A reunion of Genius and Maldoror talking each other, in the surreal style of Salvador Dali"
  text = "A lonely person flying and dreaming, in the style of Impresionism of Monet"
  image = image_gen(prompt=text).images[0]
  display(image)
  # image.save("surreal.png")


# Requieres more GPU like A100
if 1==0:
  pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16).to("cuda")
  generator = torch.Generator(device="cuda").manual_seed(0)
  prompt = "A futuristic class full of students learning AI coding in the surreal style of Salvador Dali"

  # Generate the image using the GPU
  image = pipe(
      prompt,
      guidance_scale=0.0,
      num_inference_steps=4,
      max_sequence_length=256,
      generator=generator
  ).images[0]



In [ ]:
image.save("lonely_person.png")

### Generación de audio



In [ ]:
if 1==0:
  synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts", device='cuda')
  embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
  speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)
  speech = synthesiser("We cant fail if we learn from it", forward_params={"speaker_embeddings": speaker_embedding})
  sf.write("speech.wav", speech["audio"], samplerate=speech["sampling_rate"])
  Audio("speech.wav")

## Generación de Video


In [ ]:
if 1==0:
  # Esto requiere de mucha más RAM pra ejecutarse
  # Cargar una imagen de ejemplo
  image_path = "/content/lonely_person.png"
  image = Image.open(image_path)

  pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-video-diffusion-img2vid-xt")
  # Ejecutar la generación de video con la imagen de entrada
  output = pipe(image=image)

  # Guardar el primer frame del video generado
  output.images[0].save("output_frame.png")